<a href="https://colab.research.google.com/github/Mart1n610/CardioAlert/blob/main/CardioAlert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
señalPrueba = [1, 2, 3, 4, 5, 4, 3, 2, 1]
ventanaHamming = np.hamming(len(señalPrueba))
señalVentana = señalPrueba * ventanaHamming

In [3]:
fourierft = np.fft.fft(señalVentana)
frecuenciaseñalp = 100 #frecuencia de prueba de 100 Hz
frecuenciaCardio = np.argmax(np.abs(fourierft)) * (frecuenciaseñalp / len(señalPrueba))
print(frecuenciaCardio)

0.0


In [7]:
def detectarPicos(señalPrueba):
  picos = []
  for i in range(1, len(señalPrueba)-1):
    if picos[i]>señalPrueba[i-1] and señalPrueba[i]>señalPrueba[i+1]:
      picos.append(i)
  return picos